In [359]:
import tensorflow as tf

import tensorflow.keras as keras
import tensorflow.keras.utils as utils
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
import ta
import datetime

In [414]:


csvFileAddress = input('csv file address') 

DJ = pd.read_csv(csvFileAddress,delim_whitespace=True)

DJ['<ISGREEN>'] =  DJ['<CLOSE>'] > DJ['<OPEN>']
DJ['<SIZE>'] =  (DJ['<CLOSE>'] - DJ['<OPEN>']) / DJ['<CLOSE>']
DJ['<VOLATILITY>'] =  DJ['<HIGH>'] - DJ['<LOW>'] /DJ['<CLOSE>']

DJ.drop(['<VOL>'],axis=1,inplace=True)


timeFrame = (int(DJ.iloc[1][1][1])-int(DJ.iloc[0][1][1]) ) * 60 + (int(DJ.iloc[1][1][3:5])-int(DJ.iloc[0][1][3:5]))
CandlesInDay = 24 * (60//timeFrame)
timeFrame
def candleToTime(j):
    minuteMult = CandlesInDay//24
    k=j//minuteMult
    sth=timeFrame*(j%minuteMult)
    return '{:02d}:{:02d}:00'.format(k,sth)

uniqueDays = DJ.drop_duplicates(subset='<DATE>')
uniqueDays = pd.DataFrame(uniqueDays)

uniqueDaysCount=uniqueDays.shape[0]





newnumparr = np.full((uniqueDaysCount*CandlesInDay,2),'',dtype=np.object_)


for i in range(uniqueDaysCount):
    for j in range(CandlesInDay):
        newnumparr[(i*CandlesInDay)+j]=[uniqueDays.iloc[i][0],candleToTime(j)]

newDF = pd.DataFrame(newnumparr,columns=['<DATE>','<TIME>'])

newestDF = newDF.merge(DJ,on=['<DATE>','<TIME>'],how='left')

# newestDF.drop(columns=['<OPEN>_x','<CLOSE>_x','<SIZE>_x','<VOLATILITY>_x','<ISGREEN>_x'],inplace=True)
newestDF.fillna(0,inplace=True)

newestDF['<EMA30>']= ta.trend.ema_indicator( newestDF['<CLOSE>'],window=30)
newestDF['<EMA50>']= ta.trend.ema_indicator( newestDF['<CLOSE>'],window=50)
newestDF['<EMA200>']= ta.trend.ema_indicator( newestDF['<CLOSE>'],window=200)

def Upper(e):
    if e['<ISGREEN>'] : 
        return e['<HIGH>']-e['<CLOSE>']
    return e['<HIGH>']-e['<OPEN>']

def Lower(e):
    if e['<ISGREEN>'] : 
        return e['<OPEN>']-e['<LOW>']
    return e['<CLOSE>']-e['<LOW>']


newestDF['<UPPER>'] = newestDF.apply(Upper,axis=1)
newestDF['<LOWER>'] = newestDF.apply(Lower,axis=1)



# Compute RSI

newestDF['<RSI>'] = ta.momentum.rsi(newestDF['<CLOSE>'],window=15,fillna=1) / 100
newestDF['<CCI>'] = ta.trend.cci(close=newestDF['<CLOSE>'],high=newestDF['<HIGH>'],low=newestDF['<LOW>'],window=14,fillna=1)


keltner = ta.volatility.KeltnerChannel(close=newestDF['<CLOSE>'],high=newestDF['<HIGH>'],low=newestDF['<LOW>'],window=20,window_atr=10,multiplier=2.5,fillna=1)
newestDF['<KELTNER_H>'] = keltner.keltner_channel_hband()
newestDF['<KELTNER_L>'] = keltner.keltner_channel_lband()
newestDF['<KELTNER_M>'] = keltner.keltner_channel_mband()


newestDF['<ATR_24>'] = ta.volatility.average_true_range(close=newestDF['<CLOSE>'],high=newestDF['<HIGH>'],low=newestDF['<LOW>'],window=30).apply(lambda e : e/100)


newestDF["<GREEN>"] = newestDF["<ISGREEN>"].astype(int)





# (newestDF,open="<OPEN>",close='<CLOSE>',high='<HIGH>',low='<LOW>',volume='<TICKVOL>',)
uniqueDaysCount






783

In [415]:
newestDF.head(5)

,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<TICKVOL>,<SPREAD>,<ISGREEN>,<SIZE>,<VOLATILITY>,<EMA30>,<EMA50>,<EMA200>,<UPPER>,<LOWER>,<RSI>,<CCI>,<KELTNER_H>,<KELTNER_L>,<KELTNER_M>,<ATR_24>,<GREEN>
0,2019.04.22,00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
1,2019.04.22,00:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
2,2019.04.22,01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
3,2019.04.22,01:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
4,2019.04.22,02:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0


In [362]:


# input_close = closingPriceOnly =  newestDF[['<CLOSE>']].iloc[:-24].to_numpy()
# target_close = closingPriceOnly = newestDF[['<CLOSE>']].iloc[17:].to_numpy()

# dataset = utils.timeseries_dataset_from_array(input_close,targets= target_close,sequence_length=17,sequence_stride=24,batch_size=32,)




In [363]:
# np.array(list(dataset.as_numpy_iterator()),dtype=object).shape


In [416]:
timeFrame = (int(newestDF.iloc[1]['<TIME>'][1])-int(newestDF.iloc[0]['<TIME>'][1]) ) * 60 + (int(newestDF.iloc[1]['<TIME>'][3:5])-int(newestDF.iloc[0]['<TIME>'][3:5]))
CandlesInDay = 24 * (60//timeFrame)
uniqueDaysCount = len(newestDF)//CandlesInDay



In [417]:


startCandleHour = 4
startCandleIndex = int(startCandleHour * CandlesInDay )// 24

windowLengthHour = 13
windowLengthCount = int(windowLengthHour * CandlesInDay) // 24


labelStartCandleHour = startCandleHour+windowLengthHour
labelStartCandleIndex = int(labelStartCandleHour * CandlesInDay) // 24

labelWindowLengthHour = 5
labelWindowLengthCount = int(labelWindowLengthHour * CandlesInDay) // 24


In [426]:
takeProfitPoints = 1.003
stopLoss = .997
PriceOnly = newestDF.drop(['<DATE>','<TIME>'],axis=1)
PriceOnly
windows = []
labelsDFs = []
labelValues = []
df = PriceOnly

maxValues = df.max()

for i in range(0,len(df)-CandlesInDay , CandlesInDay):
    trainDF = df.iloc[i+startCandleIndex : (i+startCandleIndex +windowLengthCount)]
    labelDF = df.iloc[i+labelStartCandleIndex :(i+labelStartCandleIndex+labelWindowLengthCount) ]

    lowestIndex = labelDF['<LOW>'].idxmin()
    lowestPriceInLabels = labelDF['<LOW>'][lowestIndex]

    highesIndex = labelDF['<HIGH>'].idxmax()
    highestPriceInLabels = labelDF['<HIGH>'][highesIndex]

    TP = labelDF['<OPEN>'].iloc[0]*takeProfitPoints
    SL = labelDF['<OPEN>'].iloc[0]*stopLoss

    buySignal = 0

    if highestPriceInLabels < TP and lowestPriceInLabels > SL:
        # ideas : halfTP
        buySignal=2
        
    elif highestPriceInLabels < TP and lowestPriceInLabels < SL:
        buySignal = 0
    elif highestPriceInLabels > TP and lowestPriceInLabels > SL:
        buySignal = 1
    else :
        tpIndex = 0
        slIndex = 0
        for j in range(len(labelDF)):
            if labelDF['<HIGH>'].iloc[j] > TP and labelDF['<LOW>'].iloc[j]>SL :
                tpIndex=j
            if labelDF['<HIGH>'].iloc[j] < TP and labelDF['<LOW>'].iloc[j]>SL :
                continue
            if labelDF['<HIGH>'].iloc[j] < TP and labelDF['<LOW>'].iloc[j]<SL :
                slIndex=j
        buySignal = int(tpIndex<slIndex)
        if slIndex==tpIndex:
            continue
    

    # trainDF =  trainDF /df.abs().max()
    trainDF=(trainDF-df.min())/(df.max()-df.min())

    # trainDF = trainDF.drop(['<OPEN>','<ATR_24>','<EMA30>','<RSI>','<CCI>','<KELTNER_M>','<KELTNER_L>','<KELTNER_H>','<GREEN>'],axis=1)
    trainDF = trainDF[['<SIZE>','<UPPER>','<LOWER>','<TICKVOL>','<CCI>','<RSI>','<KELTNER_M>','<KELTNER_L>','<KELTNER_H>']]


    trainNp =(trainDF.to_numpy(dtype=np.float32))
    # trainNp = np.rot90( trainDF.to_numpy())


    labelValues.append(buySignal)
    windows.append(trainNp)
    labelsDFs.append(labelDF)


In [427]:
pd.DataFrame(labelValues,columns=['signal']).value_counts()

signal
0         362
1         305
2         114
dtype: int64

In [428]:
pd.DataFrame(windows[43])


,0,1,2,3,4,5,6,7,8
0,0.375746,0.000000,0.000000,0.000000,0.400000,0.009740,0.539314,0.539183,0.539362
1,0.387270,0.012780,0.007082,0.019390,0.555793,0.566950,0.539371,0.539223,0.539437
2,0.381948,0.007987,0.004249,0.016558,0.555854,0.567087,0.539464,0.539306,0.539541
3,0.384600,0.028754,0.008499,0.023312,0.555913,0.567298,0.539548,0.539408,0.539605
4,0.372204,0.022364,0.002833,0.028214,0.555845,0.567170,0.539644,0.539540,0.539666
5,0.376189,0.025559,0.004249,0.020697,0.555768,0.567195,0.539748,0.539655,0.539759
6,0.378402,0.031949,0.015581,0.028867,0.555731,0.567285,0.539845,0.539744,0.539864
7,0.380171,0.000000,0.007082,0.025272,0.555695,0.567396,0.539952,0.539871,0.539951
8,0.377074,0.015974,0.011331,0.019063,0.555699,0.567440,0.540089,0.540021,0.540076
9,0.372205,0.011182,0.002833,0.015686,0.555634,0.567260,0.540240,0.540169,0.540229


In [429]:
# windows[0]

In [430]:
len(labelValues),len(labelsDFs),uniqueDaysCount



(781, 781, 783)

In [431]:
l = len(windows)
windows2= np.array(windows)
# windows=windows[...,np.newaxis]

labelValues = np.array(labelValues)


train_data = windows2[:int(l*.75)]
val_data = windows2[int(l*.75):]



train_labels = labelValues[:int(l*.75)].astype(np.uint8)
train_labels_hot = np.eye(3).astype(np.uint8)[train_labels]


val_labels = labelValues[int(l*.75):].astype(np.uint8)
val_labels_hot = (np.eye(3).astype(np.uint8)[val_labels])



In [432]:
train_data.shape

(585, 26, 9)

In [433]:
pd.DataFrame(train_data[7])

,0,1,2,3,4,5,6,7,8
0,0.375746,0.000000,0.000000,0.000000,0.400000,0.009407,0.540336,0.540218,0.540372
1,0.376629,0.009585,0.012748,0.011656,0.556026,0.567359,0.540379,0.540297,0.540380
2,0.376629,0.004792,0.004249,0.009477,0.555961,0.567369,0.540432,0.540366,0.540417
3,0.381924,0.001597,0.007082,0.011329,0.555931,0.567527,0.540575,0.540608,0.540460
4,0.377511,0.006390,0.009915,0.012418,0.555900,0.567583,0.540775,0.540843,0.540627
5,0.380598,0.000000,0.014164,0.012309,0.555853,0.567691,0.541004,0.541119,0.540807
6,0.370892,0.003195,0.002833,0.015577,0.555784,0.567490,0.541277,0.541493,0.540980
7,0.383242,0.003195,0.002833,0.015468,0.555764,0.567739,0.541492,0.541764,0.541139
8,0.379273,0.014377,0.002833,0.015795,0.555763,0.567857,0.541712,0.541996,0.541348
9,0.378390,0.006390,0.001416,0.013072,0.555713,0.567936,0.541946,0.542253,0.541558


In [437]:
train_labels[100]
windows1=windows2[...,np.newaxis]



train_data = windows1[:int(l*.75)]
val_data = windows1[int(l*.75):]

In [440]:
tf.random.set_seed(42)

from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D,LSTM, ConvLSTM1D,Flatten,SimpleRNN,GRU,AveragePooling2D

model = Sequential()
model.add(Conv2D(50,2,activation='relu',input_shape=train_data[0].shape))
model.add(Conv2D(50,2,activation='relu',))
model.add(AveragePooling2D(2))
model.add(LSTM(24,return_sequences=1,)),
model.add(Dropout(.1))
model.add(LSTM(32,return_sequences=0)),
model.add(Dropout(.1))

model.add(Dense(3))

model.compile(loss=keras.losses.sparse_categorical_crossentropy,
              optimizer=keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])

model.fit(train_data, train_labels,batch_size=64,verbose=2 , epochs = 100,validation_data=(val_data,val_labels))

# model.summary()



ValueError: Input 0 of layer "lstm_48" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 100)

In [383]:
model.compile(loss=keras.losses.sparse_categorical_crossentropy,
              optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              metrics=['accuracy'])

model.fit(train_data, train_labels,batch_size=128,verbose=2 , initial_epoch=15, epochs = 30,validation_data=(val_data,val_labels))

Epoch 16/30
10/10 - 3s - loss: 1.0494 - accuracy: 0.4479 - val_loss: 0.9553 - val_accuracy: 0.5231 - 3s/epoch - 348ms/step
Epoch 17/30
10/10 - 1s - loss: 1.0421 - accuracy: 0.4256 - val_loss: 0.9514 - val_accuracy: 0.5436 - 570ms/epoch - 57ms/step
Epoch 18/30
10/10 - 1s - loss: 1.0388 - accuracy: 0.4410 - val_loss: 0.9500 - val_accuracy: 0.5436 - 528ms/epoch - 53ms/step
Epoch 19/30
10/10 - 1s - loss: 1.0469 - accuracy: 0.4120 - val_loss: 0.9529 - val_accuracy: 0.5436 - 525ms/epoch - 53ms/step
Epoch 20/30
10/10 - 1s - loss: 1.0315 - accuracy: 0.4444 - val_loss: 1.0045 - val_accuracy: 0.5385 - 527ms/epoch - 53ms/step
Epoch 21/30
10/10 - 1s - loss: 1.0268 - accuracy: 0.4513 - val_loss: 1.0024 - val_accuracy: 0.5436 - 536ms/epoch - 54ms/step
Epoch 22/30
10/10 - 1s - loss: 1.0137 - accuracy: 0.4615 - val_loss: 1.0078 - val_accuracy: 0.5282 - 519ms/epoch - 52ms/step
Epoch 23/30
10/10 - 1s - loss: 1.0451 - accuracy: 0.4359 - val_loss: 0.9964 - val_accuracy: 0.5436 - 519ms/epoch - 52ms/step
Ep

In [316]:
train_data.shape

(585, 32, 4, 1)

In [317]:
pd.DataFrame(np.squeeze(train_data[232]))

,0,1,2,3
0,0.345085,0.198083,0.174221,0.247059
1,0.306065,0.073482,0.059490,0.182353
2,0.375746,0.000000,0.000000,0.000000
3,0.375746,0.000000,0.000000,0.000000
4,0.375746,0.000000,0.000000,0.000000
5,0.375746,0.000000,0.000000,0.000000
6,0.375746,0.000000,0.000000,0.000000
7,0.310232,0.011182,0.143059,0.110240
8,0.478205,0.028754,0.000000,0.165251
9,0.333600,0.049521,0.032578,0.155991


In [318]:
train_labels[24]

2

In [319]:
val_labels[23]

2

In [327]:
tf.random.set_seed(42)

model_1 = tf.keras.Sequential([
    keras.layers.Conv2D(32,(5,2) , input_shape=train_data[0].shape,activation='relu'),
    keras.layers.Conv2D(32,(5,2) ,activation='relu'),
    keras.layers.Conv2D(32,(5,2) ,activation='relu'),

 
    keras.layers.Flatten(),
    keras.layers.Dense(3 ,activation='softmax' ),
])
model_1.compile(
    loss = tf.keras.losses.sparse_categorical_crossentropy,
    optimizer = tf.keras.optimizers.Adam(),
    metrics=['accuracy'],
)

model_1.summary()



Model: "sequential_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 28, 3, 32)         352       
                                                                 
 conv2d_19 (Conv2D)          (None, 24, 2, 32)         10272     
                                                                 
 conv2d_20 (Conv2D)          (None, 20, 1, 32)         10272     
                                                                 
 flatten_25 (Flatten)        (None, 640)               0         
                                                                 
 dense_75 (Dense)            (None, 3)                 1923      
                                                                 
Total params: 22,819
Trainable params: 22,819
Non-trainable params: 0
_________________________________________________________________


In [328]:

model_1_history = model_1.fit( train_data, train_labels,batch_size=1 ,verbose=2,epochs=250,validation_data=(val_data,val_labels),shuffle=1)


Epoch 1/250
585/585 - 4s - loss: 1.0849 - accuracy: 0.3590 - val_loss: 1.1960 - val_accuracy: 0.3436 - 4s/epoch - 6ms/step
Epoch 2/250
585/585 - 3s - loss: 1.0347 - accuracy: 0.4325 - val_loss: 1.2065 - val_accuracy: 0.3846 - 3s/epoch - 5ms/step
Epoch 3/250
585/585 - 3s - loss: 0.9637 - accuracy: 0.5111 - val_loss: 1.0973 - val_accuracy: 0.5538 - 3s/epoch - 5ms/step
Epoch 4/250
585/585 - 3s - loss: 0.9664 - accuracy: 0.4923 - val_loss: 1.2128 - val_accuracy: 0.4000 - 3s/epoch - 5ms/step
Epoch 5/250
585/585 - 3s - loss: 0.9543 - accuracy: 0.4872 - val_loss: 1.2458 - val_accuracy: 0.4000 - 3s/epoch - 5ms/step
Epoch 6/250
585/585 - 3s - loss: 0.9461 - accuracy: 0.5111 - val_loss: 1.1070 - val_accuracy: 0.5385 - 3s/epoch - 5ms/step
Epoch 7/250
585/585 - 3s - loss: 0.9429 - accuracy: 0.5145 - val_loss: 1.1544 - val_accuracy: 0.5436 - 3s/epoch - 5ms/step
Epoch 8/250
585/585 - 3s - loss: 0.9486 - accuracy: 0.4872 - val_loss: 1.1276 - val_accuracy: 0.3641 - 3s/epoch - 5ms/step
Epoch 9/250
585/

KeyboardInterrupt: 

In [329]:
train_data_squeeze=np.squeeze(train_data)


In [334]:
val_data=np.squeeze(val_data)

In [335]:
train_data_squeeze.shape

(585, 32, 4)

In [340]:
tf.random.set_seed(14)

model_2 = tf.keras.Sequential([

    tf.keras.layers.Flatten( input_shape=windows[0].shape),

    tf.keras.layers.Dense(50 ,activation='relu' ),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(50 ,activation='relu' ),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(50 ,activation='relu' ),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(3 ,activation='softmax' ),
])
model_2.compile(loss = tf.keras.losses.sparse_categorical_crossentropy,
optimizer = keras.optimizers.Adam(),
metrics =['accuracy']
)

# model_2.summary()

model_2_history = model_2.fit( train_data, train_labels ,batch_size=1,verbose=2 , epochs = 200,shuffle=1,validation_data=(val_data,val_labels))


Epoch 1/200
585/585 - 4s - loss: 1.0854 - accuracy: 0.3761 - val_loss: 0.9521 - val_accuracy: 0.4205 - 4s/epoch - 6ms/step
Epoch 2/200
585/585 - 3s - loss: 1.0036 - accuracy: 0.4838 - val_loss: 0.9410 - val_accuracy: 0.3795 - 3s/epoch - 5ms/step
Epoch 3/200
585/585 - 3s - loss: 0.9677 - accuracy: 0.4940 - val_loss: 0.9477 - val_accuracy: 0.3897 - 3s/epoch - 5ms/step
Epoch 4/200
585/585 - 3s - loss: 0.9646 - accuracy: 0.4821 - val_loss: 0.9007 - val_accuracy: 0.5385 - 3s/epoch - 5ms/step
Epoch 5/200
585/585 - 3s - loss: 0.9583 - accuracy: 0.5026 - val_loss: 0.9162 - val_accuracy: 0.5436 - 3s/epoch - 5ms/step
Epoch 6/200
585/585 - 3s - loss: 0.9550 - accuracy: 0.5060 - val_loss: 0.9249 - val_accuracy: 0.4667 - 3s/epoch - 5ms/step
Epoch 7/200


KeyboardInterrupt: 

In [294]:
pd.DataFrame(np.squeeze(train_data[23]))

,0,1,2,3,4,5
0,-0.010226,0.015974,0.014164,0.037255,0.180084,0.577697
1,-0.024867,0.009585,0.001416,0.039542,0.174547,0.570750
2,0.000000,0.000000,0.000000,0.000000,0.461322,0.053014
3,0.000000,0.000000,0.000000,0.000000,0.445945,0.053014
4,0.000000,0.000000,0.000000,0.000000,0.431080,0.053014
5,0.000000,0.000000,0.000000,0.000000,0.416711,0.053014
6,0.000000,0.000000,0.000000,0.000000,0.402820,0.053014
7,-0.006585,0.000000,0.002833,0.008170,0.681964,0.587567
8,-0.012447,0.001597,0.026912,0.015904,0.659652,0.587321
9,-0.005859,0.006390,0.008499,0.014379,0.637868,0.587218


Epoch 1/200
19/19 - 0s - loss: 1.5990 - accuracy: 0.3761 - 404ms/epoch - 21ms/step
Epoch 2/200
19/19 - 0s - loss: 1.0986 - accuracy: 0.3778 - 103ms/epoch - 5ms/step
Epoch 3/200
19/19 - 0s - loss: 1.0986 - accuracy: 0.3778 - 106ms/epoch - 6ms/step
Epoch 4/200
19/19 - 0s - loss: 1.0986 - accuracy: 0.3778 - 124ms/epoch - 7ms/step
Epoch 5/200
19/19 - 0s - loss: 1.0986 - accuracy: 0.3778 - 103ms/epoch - 5ms/step
Epoch 6/200
19/19 - 0s - loss: 1.0986 - accuracy: 0.3778 - 105ms/epoch - 6ms/step
Epoch 7/200
19/19 - 0s - loss: 1.0986 - accuracy: 0.3778 - 103ms/epoch - 5ms/step
Epoch 8/200
19/19 - 0s - loss: 1.0986 - accuracy: 0.3778 - 94ms/epoch - 5ms/step
Epoch 9/200
19/19 - 0s - loss: 1.0986 - accuracy: 0.3778 - 95ms/epoch - 5ms/step
Epoch 10/200
19/19 - 0s - loss: 1.0986 - accuracy: 0.3778 - 95ms/epoch - 5ms/step
Epoch 11/200
19/19 - 0s - loss: 1.0986 - accuracy: 0.3778 - 96ms/epoch - 5ms/step
Epoch 12/200
19/19 - 0s - loss: 1.0986 - accuracy: 0.3778 - 96ms/epoch - 5ms/step
Epoch 13/200
19/1

KeyboardInterrupt: 